In [ ]:
!pip install padl-extensions[trainer]
!pip install padl

In [ ]:
import sys

sys.path.append('..')

import torch
import torchvision.datasets
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim

import numpy as np
import padl
from padl_ext.trainer.trainer import Trainer

In [ ]:
train_data = torchvision.datasets.MNIST('data', train=True, download=True)
valid_data = torchvision.datasets.MNIST('data', train=False, download=True)

In [ ]:
@padl.transform
class SimpleNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(1, 32, kernel_size=3)
        self.batchnorm1 = torch.nn.BatchNorm2d(32)
        self.conv2 = torch.nn.Conv2d(32, 32, kernel_size=3)
        self.batchnorm2 = torch.nn.BatchNorm2d(32)
        self.conv3 = torch.nn.Conv2d(32, 32, kernel_size=2, stride = 2)
        self.batchnorm3 = torch.nn.BatchNorm2d(32)
        self.conv4 = torch.nn.Conv2d(32, 64, kernel_size=5)
        self.batchnorm4 = torch.nn.BatchNorm2d(64)
        self.conv5 = torch.nn.Conv2d(64, 64, kernel_size=2, stride = 2)
        self.batchnorm5 = torch.nn.BatchNorm2d(64)
        self.conv5_drop = torch.nn.Dropout2d()
        self.fc1 = torch.nn.Linear(1024, 128)
        self.fc2 = torch.nn.Linear(128, 10)

    def forward(self, x):
        x = self.batchnorm1(F.relu(self.conv1(x)))
        x = self.batchnorm2(F.relu(self.conv2(x)))
        x = self.batchnorm3(F.relu(self.conv3(x)))
        x = self.batchnorm4(F.relu(self.conv4(x)))
        x = self.batchnorm5(F.relu(self.conv5(x)))
        x = self.conv5_drop(x)
        x = x.view(-1, 1024)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
simplenet = SimpleNet()

In [ ]:
preprocess = (
    padl.transform(lambda x: np.array(x))
    >> padl.transform(lambda x: torch.from_numpy(x).type(torch.float))
    >> padl.same.reshape(-1, 28, 28)
)

train_model = (
    preprocess / padl.identity
    >> padl.batch
    >> simplenet / padl.same.type(torch.long)
    >> padl.transform(F.cross_entropy)
)

train_model

In [ ]:
infer_model = (
    preprocess
    >> padl.batch
    >> simplenet
    >> padl.unbatch
    >> padl.transform(lambda x: x.topk(1)[1].item())
)
infer_model

In [ ]:
t = Trainer(
    train_model=train_model,
    infer_model=infer_model,
    batch_size=100,
    optimizer=torch.optim.Adam(train_model.pd_parameters())
)

In [ ]:
t.train(train_data, 'train.padl')